# Porównanie wyników pomiarów wydajności QuadTree and KDTree

In [17]:
# Imports geometry utilities
from utilities.Point import Point
from utilities.Rectangle import Rectangle
# Imports modules for testing, benchmarking and visualizing
from visualizer.main import Visualizer
from comparator.CaseGenerator import CaseGenerator
from tests.TestManager import TestManager
# Imports data structures (and their visualizers)
from KdTree import KdTree, KdTree_visualizer
from QuadTree import QuadTree, QuadTree_visualizer

## Spis treści:
1. Sprawdzenie poprawności działania porównywanych struktur danych
2. Pomiary wydajności dla różnych rozmiarów danych oraz rodzaju danych wejściowych
3. Generowanie wykresów
4. Pomiary dla indywidulanych przypadków pod zastosowania drzewa

## 1. Sprawdzenie poprawności działania porównywanych struktur danych

KdTree:

In [18]:
KdTree_tests = TestManager(KdTree).all_tests()

Running tests:
Test contain_point_int:
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Passed 57/57 tests.
Test contain_point_float:
++++++++++
Passed 10/10 tests.
Test points_in_rectangle_int:
++++
Passed 4/4 tests.
Test points_in_rectangle_float:
+
Passed 1/1 tests.
Passed 72/72 of all tests.


QuadTree:

In [ ]:
QuadTree_tests = TestManager(QuadTree).all_tests()

Weryfikacja:

In [ ]:
assert KdTree_tests and QuadTree_tests
print("All tests passed!")

## 2. Pomiary wydajności

2.1 Rozkład równomierny

2.2 Rozkład normalny

2.3 Rozkład "posadzka"

2.4 Rozkład klastrowy

2.5 Rozkład krzyżowy

2.6 Rozkład prostokątny

## 3. Generowanie wykresów